<h1 style="text-align:center">Machine Learning</h1>

In [1]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV

import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import DMatrix
from xgboost import cv

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from time import time
from datetime import datetime

C:\Users\Melan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


<h3 style="text-align:center">Load datasets</h3>

In [3]:
train_df = pd.read_pickle('train_df.p')
test_df = pd.read_pickle('test_df.p')
target = pd.read_pickle('target.p')

In [4]:
train_df.shape, test_df.shape, target.shape

((1458621, 16), (625134, 15), (1458621, 1))

<h3 style="text-align:center">Split-out dataset</h3>

In [14]:
# Test options and evaluation metric
num_folds = 10 
seed = 46
test_size = 0.05
scoring = 'r2'

In [15]:
X = train_df.drop(['id', 'trip_duration'], axis=1).values
Y = target.values.ravel()

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=test_size, \
                                                 random_state=seed)

In [16]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((1385689, 14), (1385689,), (72932, 14), (72932,))

<h3 style="text-align:center">Tuning Algorithm</h3>

In [17]:
# Create DMatrix to make XGBoost more efficient
xgdmat = DMatrix(X_train, y_train)
testdmat = DMatrix(X_val)